# 09_LangFuse_AB_Testing_Prompts

## Mục tiêu học tập

Hiểu cách thiết lập, chạy và phân tích các thử nghiệm A/B cho prompt bằng LangFuse để tối ưu hóa hiệu suất LLM.

## Tại sao A/B Testing cho Prompts quan trọng?

- **Tối ưu hóa hiệu suất**: So sánh các phiên bản prompt khác nhau để tìm ra cách viết tốt nhất
- **Ra quyết định dựa trên dữ liệu**: Sử dụng metrics thực tế thay vì cảm tính
- **Cải thiện liên tục**: Thử nghiệm và cải tiến prompt theo thời gian
- **Giảm bias**: Tránh việc chọn prompt dựa trên ý kiến chủ quan

## Cách LangFuse hỗ trợ A/B Testing

LangFuse cung cấp:
- **Tagging system**: Gắn thẻ các trace theo phiên bản prompt
- **Metrics tracking**: Theo dõi latency, token usage, cost
- **Dashboard phân tích**: So sánh hiệu suất giữa các phiên bản
- **Evaluation system**: Đánh giá chất lượng output tự động và thủ công

## 1. Cài đặt & Cấu hình

In [ ]:
import os
import random
import time
from datetime import datetime
from typing import Dict, List, Any

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

# Kiểm tra API keys
if not os.getenv("ANTHROPIC_API_KEY"):
    raise ValueError("ANTHROPIC_API_KEY environment variable is required")

if not os.getenv("LANGFUSE_SECRET_KEY") or not os.getenv("LANGFUSE_PUBLIC_KEY"):
    raise ValueError("LANGFUSE_SECRET_KEY and LANGFUSE_PUBLIC_KEY environment variables are required")

print("✅ Environment variables configured successfully")

## 2. Khởi tạo LangFuse và LLM

In [ ]:
# Khởi tạo LangFuse
langfuse = Langfuse()

# Khởi tạo LLM
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0.1,
    max_tokens=500
)

print("✅ LangFuse và ChatAnthropic initialized successfully")

## 3. Định nghĩa Use Case: A/B Testing cho Prompt Tóm tắt Văn bản

Chúng ta sẽ thử nghiệm hai phiên bản prompt khác nhau cho việc tóm tắt văn bản:
- **Prompt A**: Phong cách trực tiếp, ngắn gọn
- **Prompt B**: Phong cách có cấu trúc, chi tiết hơn

In [ ]:
# Định nghĩa hai phiên bản prompt
PROMPT_A = """
Hãy tóm tắt văn bản sau trong 2-3 câu ngắn gọn:

{text}
"""

PROMPT_B = """
Bạn là một chuyên gia tóm tắt văn bản. Hãy phân tích và tóm tắt văn bản sau theo cấu trúc:

**Chủ đề chính:** [1 câu]
**Điểm quan trọng:** [2-3 điểm chính]
**Kết luận:** [1 câu]

Văn bản cần tóm tắt:
{text}
"""

# Tạo prompt templates
template_a = ChatPromptTemplate.from_template(PROMPT_A)
template_b = ChatPromptTemplate.from_template(PROMPT_B)

print("✅ Đã định nghĩa hai phiên bản prompt A và B")

## 4. Dữ liệu Test Samples

In [ ]:
# Dữ liệu mẫu để test
test_texts = [
    """
Trí tuệ nhân tạo (AI) đang thay đổi cách chúng ta làm việc và sống. 
Từ việc tự động hóa các tác vụ đơn giản đến việc hỗ trợ ra quyết định phức tạp, 
AI mang lại nhiều lợi ích. Tuy nhiên, cũng có những lo ngại về việc mất việc làm 
và vấn đề đạo đức trong việc sử dụng AI. Để tận dụng tối đa AI, 
chúng ta cần phát triển kỹ năng phù hợp và xây dựng khung pháp lý phù hợp.
    """.strip(),
    
    """
Biến đổi khí hậu là một trong những thách thức lớn nhất của thế kỷ 21. 
Nhiệt độ toàn cầu tăng do hoạt động của con người, đặc biệt là việc đốt nhiên liệu hóa thạch. 
Điều này dẫn đến băng tan, mực nước biển dâng và thời tiết cực đoan. 
Để giải quyết vấn đề này, cần có sự hợp tác quốc tế, 
đầu tư vào năng lượng tái tạo và thay đổi lối sống bền vững.
    """.strip(),
    
    """
Giáo dục trực tuyến đã trở nên phổ biến, đặc biệt sau đại dịch COVID-19. 
Nó mang lại sự linh hoạt về thời gian và địa điểm học tập, 
giúp nhiều người tiếp cận được kiến thức chất lượng. 
Tuy nhiên, giáo dục trực tuyến cũng có những hạn chế như thiếu tương tác trực tiếp, 
cần kỷ luật tự học cao và vấn đề công nghệ. 
Tương lai giáo dục có thể là sự kết hợp giữa trực tuyến và trực tiếp.
    """.strip()
]

print(f"✅ Chuẩn bị {len(test_texts)} văn bản test")

## 5. Triển khai A/B Testing với LangFuse Tracking

In [ ]:
def run_ab_test_with_tracking(text: str, prompt_version: str) -> Dict[str, Any]:
    """
    Chạy một test với tracking đầy đủ bằng LangFuse
    """
    # Tạo callback handler với tags
    callback_handler = CallbackHandler(
        tags=[f"prompt_version_{prompt_version}", "ab_testing", "summarization"],
        user_id="ab_test_user",
        session_id=f"ab_test_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    
    # Chọn prompt template dựa trên version
    if prompt_version == "A":
        template = template_a
    else:
        template = template_b
    
    # Bắt đầu đo thời gian
    start_time = time.time()
    
    try:
        # Tạo chain với callback
        chain = template | llm
        
        # Gọi LLM
        result = chain.invoke(
            {"text": text},
            config={"callbacks": [callback_handler]}
        )
        
        # Tính thời gian xử lý
        processing_time = time.time() - start_time
        
        return {
            "prompt_version": prompt_version,
            "input_text": text,
            "output": result.content,
            "processing_time": processing_time,
            "success": True,
            "error": None
        }
        
    except Exception as e:
        processing_time = time.time() - start_time
        return {
            "prompt_version": prompt_version,
            "input_text": text,
            "output": None,
            "processing_time": processing_time,
            "success": False,
            "error": str(e)
        }

print("✅ Đã định nghĩa function run_ab_test_with_tracking")

## 6. Chạy A/B Testing

In [ ]:
# Chạy A/B test cho tất cả các văn bản
results = []

print("🚀 Bắt đầu chạy A/B testing...")
print("=" * 50)

for i, text in enumerate(test_texts):
    print(f"\n📝 Test văn bản #{i+1}")
    
    # Test với cả hai phiên bản prompt
    for version in ["A", "B"]:
        print(f"  ⏳ Đang test Prompt {version}...")
        
        result = run_ab_test_with_tracking(text, version)
        results.append(result)
        
        if result["success"]:
            print(f"  ✅ Prompt {version}: {result['processing_time']:.2f}s")
        else:
            print(f"  ❌ Prompt {version}: Error - {result['error']}")
        
        # Tạm dừng để tránh rate limiting
        time.sleep(1)

print(f"\n🎉 Hoàn thành A/B testing với {len(results)} kết quả")

## 7. Phân tích Kết quả A/B Testing

In [ ]:
# Phân tích kết quả
def analyze_ab_results(results: List[Dict[str, Any]]):
    """
    Phân tích kết quả A/B testing
    """
    # Phân chia kết quả theo prompt version
    results_a = [r for r in results if r["prompt_version"] == "A" and r["success"]]
    results_b = [r for r in results if r["prompt_version"] == "B" and r["success"]]
    
    print("📊 PHÂN TÍCH KẾT QUẢ A/B TESTING")
    print("=" * 50)
    
    # Thống kê cơ bản
    print(f"\n📈 Thống kê cơ bản:")
    print(f"  • Prompt A: {len(results_a)} tests thành công")
    print(f"  • Prompt B: {len(results_b)} tests thành công")
    
    if results_a and results_b:
        # So sánh thời gian xử lý
        avg_time_a = sum(r["processing_time"] for r in results_a) / len(results_a)
        avg_time_b = sum(r["processing_time"] for r in results_b) / len(results_b)
        
        print(f"\n⏱️ Thời gian xử lý trung bình:")
        print(f"  • Prompt A: {avg_time_a:.2f}s")
        print(f"  • Prompt B: {avg_time_b:.2f}s")
        
        if avg_time_a < avg_time_b:
            print(f"  🏆 Prompt A nhanh hơn {avg_time_b - avg_time_a:.2f}s")
        else:
            print(f"  🏆 Prompt B nhanh hơn {avg_time_a - avg_time_b:.2f}s")
        
        # So sánh độ dài output
        avg_len_a = sum(len(r["output"]) for r in results_a) / len(results_a)
        avg_len_b = sum(len(r["output"]) for r in results_b) / len(results_b)
        
        print(f"\n📏 Độ dài output trung bình:")
        print(f"  • Prompt A: {avg_len_a:.0f} ký tự")
        print(f"  • Prompt B: {avg_len_b:.0f} ký tự")
    
    return results_a, results_b

# Chạy phân tích
results_a, results_b = analyze_ab_results(results)

## 8. Hiển thị Kết quả Chi tiết

In [ ]:
# Hiển thị kết quả chi tiết để so sánh chất lượng
print("🔍 SO SÁNH CHI TIẾT KẾT QUẢ")
print("=" * 70)

for i in range(min(len(results_a), len(results_b))):
    print(f"\n📝 Văn bản #{i+1}:")
    print(f"Input: {results_a[i]['input_text'][:100]}...")
    
    print(f"\n🅰️ Prompt A Output ({results_a[i]['processing_time']:.2f}s):")
    print(f"{results_a[i]['output']}")
    
    print(f"\n🅱️ Prompt B Output ({results_b[i]['processing_time']:.2f}s):")
    print(f"{results_b[i]['output']}")
    
    print("\n" + "-" * 70)

## 9. Thêm Manual Evaluation với LangFuse

In [ ]:
# Thêm evaluation scores thủ công cho một số kết quả
def add_manual_evaluation(result: Dict[str, Any], quality_score: float, relevance_score: float):
    """
    Thêm đánh giá thủ công vào LangFuse
    """
    try:
        # Tạo evaluation entry
        langfuse.score(
            name="quality_score",
            value=quality_score,
            comment=f"Manual quality evaluation for prompt version {result['prompt_version']}"
        )
        
        langfuse.score(
            name="relevance_score", 
            value=relevance_score,
            comment=f"Manual relevance evaluation for prompt version {result['prompt_version']}"
        )
        
        return True
    except Exception as e:
        print(f"Error adding evaluation: {e}")
        return False

# Thêm một số đánh giá mẫu
print("📋 Thêm manual evaluations...")

# Đánh giá mẫu (trong thực tế sẽ do con người đánh giá)
sample_evaluations = [
    {"quality": 8.5, "relevance": 9.0},  # Prompt A - Text 1
    {"quality": 9.2, "relevance": 8.8},  # Prompt B - Text 1
    {"quality": 7.8, "relevance": 8.5},  # Prompt A - Text 2
    {"quality": 8.9, "relevance": 9.1},  # Prompt B - Text 2
]

for i, eval_data in enumerate(sample_evaluations[:len(results)]):
    if i < len(results):
        success = add_manual_evaluation(
            results[i], 
            eval_data["quality"], 
            eval_data["relevance"]
        )
        if success:
            print(f"  ✅ Added evaluation for result #{i+1}")

print("✅ Manual evaluations added to LangFuse")

## 10. Tổng hợp và Kết luận

In [ ]:
# Tổng hợp kết quả cuối cùng
def generate_final_report(results_a: List[Dict], results_b: List[Dict]):
    """
    Tạo báo cáo tổng hợp A/B testing
    """
    print("📊 BÁO CÁO TỔNG HỢP A/B TESTING")
    print("=" * 50)
    
    if not results_a or not results_b:
        print("❌ Không đủ dữ liệu để so sánh")
        return
    
    # Metrics comparison
    avg_time_a = sum(r["processing_time"] for r in results_a) / len(results_a)
    avg_time_b = sum(r["processing_time"] for r in results_b) / len(results_b)
    avg_len_a = sum(len(r["output"]) for r in results_a) / len(results_a)
    avg_len_b = sum(len(r["output"]) for r in results_b) / len(results_b)
    
    print(f"\n🔍 Kết quả so sánh:")
    print(f"\n⏱️ Tốc độ:")
    if avg_time_a < avg_time_b:
        print(f"  🏆 Prompt A thắng (nhanh hơn {avg_time_b - avg_time_a:.2f}s)")
    else:
        print(f"  🏆 Prompt B thắng (nhanh hơn {avg_time_a - avg_time_b:.2f}s)")
    
    print(f"\n📏 Độ chi tiết:")
    if avg_len_b > avg_len_a:
        print(f"  🏆 Prompt B chi tiết hơn ({avg_len_b - avg_len_a:.0f} ký tự)")
    else:
        print(f"  🏆 Prompt A ngắn gọn hơn ({avg_len_a - avg_len_b:.0f} ký tự)")
    
    print(f"\n🎯 Khuyến nghị:")
    if avg_time_a < avg_time_b and avg_len_a > 0:
        print("  • Prompt A phù hợp khi cần tốc độ và tính ngắn gọn")
        print("  • Prompt B phù hợp khi cần độ chi tiết và cấu trúc")
    else:
        print("  • Prompt B có vẻ toàn diện hơn với cấu trúc rõ ràng")
        print("  • Prompt A phù hợp cho use case cần tóm tắt nhanh")
    
    print(f"\n💡 Hướng dẫn sử dụng LangFuse Dashboard:")
    print("  1. Truy cập LangFuse dashboard")
    print("  2. Filter theo tags: 'prompt_version_A' và 'prompt_version_B'")
    print("  3. So sánh metrics: latency, token usage, costs")
    print("  4. Xem detailed traces để hiểu chi tiết")
    print("  5. Phân tích evaluation scores để đánh giá chất lượng")

# Tạo báo cáo cuối cùng
generate_final_report(results_a, results_b)

## 11. Advanced: Automated Evaluation với LangFuse

In [ ]:
# Ví dụ về automated evaluation
def automated_evaluation(output_text: str, input_text: str) -> Dict[str, float]:
    """
    Đánh giá tự động chất lượng output
    (Đây là ví dụ đơn giản, trong thực tế có thể sử dụng các models chuyên dụng)
    """
    # Metrics đơn giản
    compression_ratio = len(output_text) / len(input_text)
    
    # Đánh giá dựa trên độ nén và structure
    structure_score = 8.0 if "**" in output_text else 6.0  # Có structure formatting
    conciseness_score = max(1.0, 10.0 - (compression_ratio * 10))  # Penalty for too long
    
    return {
        "compression_ratio": compression_ratio,
        "structure_score": structure_score,
        "conciseness_score": conciseness_score
    }

# Chạy automated evaluation cho một số kết quả
print("🤖 AUTOMATED EVALUATION")
print("=" * 40)

for i, result in enumerate(results[:4]):  # Evaluate first 4 results
    if result["success"]:
        eval_scores = automated_evaluation(result["output"], result["input_text"])
        
        print(f"\nResult #{i+1} (Prompt {result['prompt_version']}):")
        print(f"  📊 Compression ratio: {eval_scores['compression_ratio']:.2f}")
        print(f"  🏗️ Structure score: {eval_scores['structure_score']:.1f}/10")
        print(f"  ✂️ Conciseness score: {eval_scores['conciseness_score']:.1f}/10")
        
        # Có thể gửi scores này lên LangFuse
        try:
            langfuse.score(
                name="automated_structure",
                value=eval_scores['structure_score'],
                comment=f"Automated structure evaluation for prompt {result['prompt_version']}"
            )
        except Exception as e:
            print(f"  ⚠️ Could not upload score: {e}")

print("\n✅ Automated evaluation completed")

## 12. Tài liệu Tham khảo

### LangFuse Documentation
- **A/B Testing Guide**: https://langfuse.com/docs/experimentation
- **Prompt Management**: https://langfuse.com/docs/prompts
- **Evaluation & Scoring**: https://langfuse.com/docs/scores
- **Tags & Filtering**: https://langfuse.com/docs/tracing-features/tags
- **Dashboard Analytics**: https://langfuse.com/docs/analytics

### Best Practices
- **Experimentation Best Practices**: https://langfuse.com/docs/experimentation/best-practices
- **Prompt Engineering**: https://langfuse.com/docs/prompts/best-practices
- **LLM Observability**: https://langfuse.com/docs/tracing

### Integration Guides
- **LangChain Integration**: https://langfuse.com/docs/integrations/langchain
- **Python SDK**: https://langfuse.com/docs/sdk/python

## 13. Kết luận & Bước tiếp theo

### Những gì đã học được:

✅ **Thiết lập A/B Testing cho Prompts**
- Định nghĩa nhiều phiên bản prompt cho cùng một tác vụ
- Sử dụng tags để phân biệt các phiên bản trong LangFuse
- Tracking metrics như latency, token usage, output quality

✅ **Phân tích dữ liệu với LangFuse**
- So sánh hiệu suất giữa các prompt versions
- Sử dụng dashboard để visualize kết quả
- Manual và automated evaluation

✅ **Ra quyết định dựa trên dữ liệu**
- Chọn prompt version tối ưu dựa trên metrics thực tế
- Hiểu trade-offs giữa tốc độ và chất lượng
- Cân nhắc use case specific requirements

### Bước tiếp theo:

🚀 **Nâng cao A/B Testing**
- Test nhiều prompt variations hơn (A/B/C/D testing)
- Thử nghiệm với different model parameters (temperature, max_tokens)
- A/B test different LLM models

🎯 **Automated Evaluation Pipeline**
- Tích hợp evaluation models chuyên dụng
- Set up continuous evaluation cho production prompts
- Build automated decision making based on evaluation results

📊 **Production Monitoring**
- Monitor prompt performance in production
- Set up alerts cho performance degradation
- Implement gradual rollout strategies

### Key Takeaways:

1. **Always measure**: Đừng dựa vào cảm tính, hãy đo lường thực tế
2. **Use proper tagging**: Tags giúp filter và phân tích dữ liệu dễ dàng
3. **Consider multiple metrics**: Không chỉ accuracy, mà còn latency, cost, user experience
4. **Iterative improvement**: A/B testing là quá trình liên tục, không phải one-time activity

LangFuse làm cho việc A/B testing prompts trở nên đơn giản và có hệ thống, giúp bạn tối ưu hóa LLM applications một cách khoa học! 🎉